In [ ]:
!pip install openai transformers pdfplumber numpy pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import openai
openai.organization = "<your_organization_key>"
openai.api_key = "<your_api_key>"

In [ ]:
import pdfplumber

pdf_path = "essayonessaywriting.pdf"
text = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text += page.extract_text()

def split_text_into_chunks(text, chunk_size=100):
    words = text.split()
    text_chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i+chunk_size])
        text_chunks.append(chunk)
    return text_chunks

text_chunks = split_text_into_chunks(text)


In [ ]:
import numpy as np

def create_embedding(chunk):
    response = openai.Embedding.create(
            input=chunk,
            model="text-embedding-ada-002"
        )
    return response["data"][0]["embedding"]

def create_embeddings(text_chunks):
    embeddings = []

    for chunk in text_chunks:
        response = openai.Embedding.create(
            input=chunk,
            model="text-embedding-ada-002"
        )
        embedding = response["data"][0]["embedding"]
        embeddings.append(embedding)

    return np.array(embeddings)

embeddings = create_embeddings(text_chunks)

In [ ]:
import pinecone

pinecone.init(api_key="<pinecone_api_key>", environment="us-east4-gcp")
index_name = "semanticsearch"
pinecone.create_index(name=index_name, metric="cosine", dimension=1536)


In [ ]:
import uuid
pinecone_index = pinecone.Index(index_name=index_name)

for chunk in text_chunks:
    id = uuid.uuid4().hex
    embedding = create_embedding(chunk)
    pinecone_index.upsert([(id, embedding, {
                "text": chunk})])

In [ ]:
query = "how to relate ideas?"
response = openai.Embedding.create(
    input=query,
    model="text-embedding-ada-002"
)
query_embedding = response["data"][0]["embedding"]

In [ ]:
num_results = 5
index = pinecone.Index("semanticsearch")
results = index.query(query_embedding, k=num_results, top_k=5, include_metadata=True)
print("Search results for the query:", query)
for match in results["matches"]:
    print(f"Text chunk: {match['metadata']['text']}")
    print(f"Similarity score: {match['score']}")
    print("\n")


Search results for the query: how to relate ideas?
Text chunk: other topic and then coming back to the first. If you use such words as ‘therefore’, make sure that what you say next really does follow from what you have just said. Think about your audience: who are you writing for? It is often better to think of your reader not as your lecturer or tutor, but as someone who may not know very much about the question. If you have your tutor in mind as the reader, it is all too easy to fall into the trap of thinking, ‘If it’s not quite clear, Dr so-and-so will understand what
Similarity score: 0.819455445


Text chunk: I’m trying to say and fill in the gaps.’ If, however, you think of yourself as having to explain a topic to another student on the course who perhaps hasn’t done the same reading as you, it forces you to think more carefully about how to introduce and explain the topic. At various strategic points in the essay, remind your reader where you are in your overall structure by usi